# Perspective 1 Heatmap Visualizations

In [7]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from IPython.display import HTML
HTML('<style>.input{display:none !important}</style>')
def load_f1_data():
    """Load and prepare the F1 data"""
    try:
        df = pd.read_csv('merged_f1_data_1994_2022.csv')
    except FileNotFoundError:
        return None
    
    # DNF DSQ etc
    df['FinPos'] = df['FinPos'].astype(str)
    return df
df = load_f1_data()

In [9]:
def create_heatmap_data(df, selected_year=None):
    """Create heatmap data for starting vs finishing positions"""
    
    if selected_year and selected_year != 'Total':
        df_filtered = df[df['Year'] == selected_year].copy()
        title_suffix = f" - {selected_year}"
    else:
        df_filtered = df.copy()
        title_suffix = " - All Years (1994-2022)"
    
    # Grouping all non-finishing positions
    df_filtered = df_filtered.copy()
    non_finish_codes = ['NC', 'DNF', 'DSQ', 'DNS', 'EX', 'WD']

    df_filtered.loc[:, 'FinPos_Display'] = df_filtered['FinPos'].copy()
    
    # non-finish > DNF
    for code in non_finish_codes:
        df_filtered.loc[:, 'FinPos_Display'] = df_filtered['FinPos_Display'].replace(code, 'DNF')

    max_start_pos = min(df_filtered['Pos'].max(), 26)
    numeric_positions = []
    non_numeric_positions = []
    
    for pos in df_filtered['FinPos_Display'].unique():
        try:
            num_pos = int(float(pos))
            if 1 <= num_pos <= 30:
                numeric_positions.append(num_pos)
        except (ValueError, TypeError):
            if pos != 'nan':
                non_numeric_positions.append(pos)

    numeric_positions = sorted(set(numeric_positions))
    finish_categories = numeric_positions + sorted(set(non_numeric_positions))
    start_categories = list(range(1, max_start_pos + 1))
    heatmap_data = np.zeros((len(finish_categories), len(start_categories)))
    for _, row in df_filtered.iterrows():
        start_pos = row['Pos']
        finish_pos = row['FinPos_Display']
        if start_pos in start_categories:
            start_idx = start_categories.index(start_pos)
            try:
                finish_pos_int = int(float(finish_pos))
                if finish_pos_int in finish_categories:
                    finish_idx = finish_categories.index(finish_pos_int)
                    heatmap_data[finish_idx, start_idx] += 1
            except (ValueError, TypeError):
                if finish_pos in finish_categories:
                    finish_idx = finish_categories.index(finish_pos)
                    heatmap_data[finish_idx, start_idx] += 1
    return heatmap_data, start_categories, finish_categories, title_suffix, len(df_filtered)


In [10]:
def create_plotly_heatmap(df, selected_year=None):
    """Create an interactive heatmap using Plotly"""
    
    heatmap_data, start_categories, finish_categories, title_suffix, total_records = create_heatmap_data(df, selected_year)
    hover_text = []
    for i in range(len(finish_categories)):
        hover_row = []
        for j in range(len(start_categories)):
            count = int(heatmap_data[i, j])
            percentage = (count / total_records * 100) if total_records > 0 else 0
            hover_row.append(f'Starting: P{start_categories[j]}<br>Finishing: {finish_categories[i]}<br>Count: {count}<br>Percentage: {percentage:.1f}%')
        hover_text.append(hover_row)
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_data,
        x=[f"P{pos}" for pos in start_categories],
        y=[str(pos) for pos in finish_categories],
        colorscale='RdYlBu_r',
        hoverongaps=False,
        hovertemplate='%{customdata}<extra></extra>',
        customdata=hover_text,
        colorbar=dict(title="Number of Occurrences")
    ))
    
    fig.update_layout(
        title={
            'text': f'F1 Starting vs Finishing Position Heatmap{title_suffix}<br><sub>Total Records: {total_records:,}</sub>',
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 16}
        },
        xaxis_title='Starting Position',
        yaxis_title='Finishing Position',
        xaxis=dict(side='bottom'),
        yaxis=dict(autorange='reversed'),
        width=1000,
        height=700,
        font=dict(size=12)
    )
    
    return fig

In [11]:
def create_interactive_heatmap_widget(df):
    """Create an interactive widget with dropdown for year selection"""
    
    if df is None:
        return
    years = sorted(df['Year'].unique())
    year_options = [('All Years (Total)', 'Total')] + [(str(year), year) for year in years]
    #dropdown
    year_dropdown = widgets.Dropdown(
        options=year_options,
        value='Total',
        description='Select Year:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='200px')
    )

    info_output = widgets.Output()

    plot_output = widgets.Output()
    
    def update_heatmap(change):
        """Update heatmap when dropdown changes"""
        selected_year = change['new']
        year_value = None if selected_year == 'Total' else selected_year
        with info_output:
            clear_output(wait=True)
            if selected_year == 'Total':
                records_count = len(df)
                print(f"Showing data for ALL YEARS: {records_count:,} total records")
            else:
                year_data = df[df['Year'] == selected_year]
                records_count = len(year_data)
                races_count = year_data['Grand Prix'].nunique()
                print(f"Showing data for {selected_year}: {records_count} records from {races_count} races")
        with plot_output:
            clear_output(wait=True)
            try:
                fig = create_plotly_heatmap(df, year_value)
                display(fig)
            except Exception as e:
                pass
    year_dropdown.observe(update_heatmap, names='value')
    
    with plot_output:
        try:
            fig = create_plotly_heatmap(df, None)
            display(fig)
        except Exception as e:
            pass
    widget_box = widgets.VBox([
        widgets.HTML("<h3>🏎️ F1 Starting vs Finishing Position Analysis</h3>"),
        year_dropdown,
        info_output,
        plot_output
    ])
    
    display(widget_box)


In [12]:
def analyze_position_data(df):
    """Analyze interesting patterns in the data"""
    if df is None:
        return
    finish_counts = df['FinPos'].value_counts()
    numeric_finishes = 0
    total_dnf_count = 0
    non_finish_codes = ['DNF', 'DSQ', 'NC', 'DNS', 'EX', 'WD']
    for pos, count in finish_counts.items():
        try:
            int(float(pos))
            numeric_finishes += count
        except (ValueError, TypeError):
            if pos in non_finish_codes:
                total_dnf_count += count
    df_numeric = df.copy()
    df_numeric['FinPos_Numeric'] = pd.to_numeric(df_numeric['FinPos'], errors='coerce')
    df_classified = df_numeric.dropna(subset=['FinPos_Numeric'])
    
    if len(df_classified) > 0:
        df_classified['Position_Change'] = df_classified['Pos'] - df_classified['FinPos_Numeric']
        pole_data = df_classified[df_classified['Pos'] == 1]
        if len(pole_data) > 0:
            pole_wins = len(pole_data[pole_data['FinPos_Numeric'] == 1])

In [13]:
def show_yearly_stats(df):
    """Show quick statistics by year"""
    if df is None:
        return
        
    yearly_stats = df.groupby('Year').agg({
        'DriverCode': 'count',
        'Grand Prix': 'nunique'
    }).rename(columns={'DriverCode': 'Total_Records', 'Grand Prix': 'Races'}).copy()
    yearly_dnf = []
    for year in sorted(df['Year'].unique()):
        year_data = df[df['Year'] == year]
        total = len(year_data)
        dnf_count = len(year_data[~year_data['FinPos'].str.isnumeric().fillna(False)])
        dnf_rate = (dnf_count / total * 100) if total > 0 else 0
        yearly_dnf.append(dnf_rate)
    
    yearly_stats.loc[:, 'DNF_Rate'] = yearly_dnf
create_interactive_heatmap_widget(df)
analyze_position_data(df)
show_yearly_stats(df)

/tmp/ipykernel_22174/1594348308.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

